In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Loading the dataset
csv_file_path = 'wavelet_metrics.csv'  
data = pd.read_csv(csv_file_path)

# Separating the features and labels
X = data.drop(['Data', 'label'], axis=1)  
y = data['label']  # Extract labels (0 or 1)

class_0 = data[data['label'] == 0]
class_1 = data[data['label'] == 1]

# Split data for training and testing

train_0, test_0 = train_test_split(class_0, test_size=0.5, random_state=0)
train_1, test_1 = train_test_split(class_1, test_size=0.5, random_state=0)

# Concatenated the training and testing data for both classes

train_data = pd.concat([train_0, train_1])
test_data = pd.concat([test_0, test_1])

# Suffling the data
train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

# Extracting features and labels from the training and testing sets
X_train = train_data.drop(['Data', 'label'], axis=1)
y_train = train_data['label']
X_test = test_data.drop(['Data', 'label'], axis=1)
y_test = test_data['label']

# Standardizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# LDA for dimensionality reduction
lda = LDA(n_components=1)  # Binary classification, so 1 component is sufficient
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

#Training an SVM classifier on the LDA-transformed data
svm = SVC( C = 1, kernel='rbf', random_state=0, gamma=0.02)  
svm.fit(X_train_lda, y_train)

# Predict and evaluate the SVM classifier
y_pred = svm.predict(X_test_lda)

# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Calculate Sensitivity and Specificity
sensitivity = tp / (tp + fn)  # True Positive Rate
specificity = tn / (tn + fp)  # True Negative Rate

# Print evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print(f"\nSensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Confusion Matrix:
[[400   0]
 [  1 399]]

Sensitivity: 1.00
Specificity: 1.00
Accuracy: 1.00


In [ ]:
param_grid = {
    'C': [0.01, 0.1, 1, 2, 3, 5, 10, 11, 20, 30, 40, 50, 100, 200],      # Regularization parameter
    'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 1],  # RBF Kernel coefficient
}

# Use StratifiedKFold for balanced cross-validation splits
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Defiing the SVM with RBF kernel
svm = SVC(kernel='rbf', random_state=0)

# Perform GridSearchCV to find the optimal hyperparameters
grid_search = GridSearchCV(svm, param_grid, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_lda, y_train)

# Print the best hyperparameters
print(f"Best Parameters: {grid_search.best_params_}")

# Training the final SVM model with the best parameters
best_svm = grid_search.best_estimator_
best_svm.fit(X_train_lda, y_train)

# Predict on the test set
y_pred = best_svm.predict(X_test_lda)

# Calculating the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Calculate Sensitivity and Specificity
sensitivity = tp / (tp + fn)  # True Positive Rate
specificity = tn / (tn + fp)  # True Negative Rate

# Evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print(f"\nSensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")


Fitting 10 folds for each of 112 candidates, totalling 1120 fits
Best Parameters: {'C': 0.01, 'gamma': 0.5}
Confusion Matrix:
[[400   0]
 [  1 399]]

Sensitivity: 1.00
Specificity: 1.00
Accuracy: 1.00
